In [1]:
!pip install streamlit pandas sqlalchemy psycopg2-binary requests

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.0 MB/s eta 0:00:00


In [4]:
import requests

API_KEY = "61c0fb7393msh2cf7194e33013a4p1f0be2jsn777143b23bb7"   # <-- put your RapidAPI key here

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live"
headers = {
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com",
    "x-rapidapi-key": API_KEY
}

response = requests.get(url, headers=headers)

print("Status Code:", response.status_code)
if response.status_code == 200:
    data = response.json()
    print("✅ Success! Top-level keys:", data.keys())
else:
    print("❌ Error:", response.text)

Status Code: 200
✅ Success! Top-level keys: dict_keys(['typeMatches', 'filters', 'appIndex', 'responseLastUpdated'])


In [5]:
import sqlite3

# Connect to database (creates file if not exist)
conn = sqlite3.connect("../cricbuzz.db")
cursor = conn.cursor()

# Create a simple table for matches
cursor.execute("""
CREATE TABLE IF NOT EXISTS live_matches (
    match_id TEXT PRIMARY KEY,
    series_name TEXT,
    team1 TEXT,
    team2 TEXT,
    status TEXT
)
""")

# Insert data into DB
for match in data.get("matches", []):
    match_id = match.get("matchId")
    info = match.get("matchInfo", {})
    team1 = info.get("team1", {}).get("teamName", "")
    team2 = info.get("team2", {}).get("teamName", "")
    series = match.get("seriesName", "")
    status = info.get("status", "")

    cursor.execute("""
    INSERT OR REPLACE INTO live_matches 
    (match_id, series_name, team1, team2, status)
    VALUES (?, ?, ?, ?, ?)
    """, (match_id, series, team1, team2, status))

conn.commit()
conn.close()
print("✅ Data saved into cricbuzz.db")

✅ Data saved into cricbuzz.db


In [1]:
import requests, os

API_KEY = os.getenv("RAPIDAPI_KEY") or "61c0fb7393msh2cf7194e33013a4p1f0be2jsn777143b23bb7"  # temp fallback

URL = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live"
HEADERS = {
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com",
    "x-rapidapi-key": API_KEY
}

resp = requests.get(URL, headers=HEADERS)
print("Status Code:", resp.status_code)
print("Sample:", resp.text[:300])

Status Code: 200
Sample: {
  "typeMatches": [{
    "matchType": "International",
    "seriesMatches": [{
      "seriesAdWrapper": {
        "seriesId": 10647,
        "seriesName": "Netherlands tour of Bangladesh 2025",
        "matches": [{
          "matchInfo": {
            "matchId": 131119,
            "seriesId": 106


In [7]:
# Peek into structure to confirm paths
for tm in data.get("typeMatches", []):
    print("matchType:", tm.get("matchType"))
    for s in tm.get("seriesMatches", []):
        series = s.get("seriesAdWrapper", {})
        print("  seriesName:", series.get("seriesName"))
        print("  matches count:", len(series.get("matches", [])))
        break  # just first series to keep output short
    break  # just first matchType

In [5]:
import os, sqlite3

DB_PATH = r"C:\Users\susen\Desktop\CricbuzzProject\cricbuzz.db"  # full path
conn = sqlite3.connect(DB_PATH)

# Show schema
df = pd.read_sql_query("PRAGMA table_info(live_matches)", conn)
print("📌 Table Schema:")
print(df)

# Show rows
df2 = pd.read_sql_query("SELECT * FROM live_matches LIMIT 30", conn)
display(df2)

conn.close()

📌 Table Schema:
    cid            name     type  notnull dflt_value  pk
0     0        match_id     TEXT        0       None   1
1     1     series_name     TEXT        0       None   0
2     2           team1     TEXT        0       None   0
3     3           team2     TEXT        0       None   0
4     4          status     TEXT        0       None   0
5     5      match_desc     TEXT        0       None   0
6     6        start_ts  INTEGER        0       None   0
7     7      venue_name     TEXT        0       None   0
8     8      venue_city     TEXT        0       None   0
9     9   venue_country     TEXT        0       None   0
10   10    match_format     TEXT        0       None   0
11   11          winner     TEXT        0       None   0
12   12    victory_type     TEXT        0       None   0
13   13  victory_margin  INTEGER        0       None   0
14   14     is_complete  INTEGER        0       None   0
15   15      updated_at     TEXT        0       None   0


,match_id,series_name,team1,team2,status,match_desc,start_ts,venue_name,venue_city,venue_country,match_format,winner,victory_type,victory_margin,is_complete,updated_at
0,131119,Netherlands tour of Bangladesh 2025,Bangladesh,Netherlands,Rain stops play,None,None,None,None,None,None,None,None,None,None,None
1,122962,"Sri Lanka tour of Zimbabwe, 2025",Zimbabwe,Sri Lanka,Sri Lanka won by 4 wkts,None,None,None,None,None,None,None,None,None,None,None
2,130762,Kerala Cricket League 2025,Aries Kollam Sailors,Kochi Blue Tigers,Innings Break,None,None,None,None,None,None,None,None,None,None,None
3,130756,Kerala Cricket League 2025,Adani Trivandrum Royals,Alleppey Ripples,Adani Trivandrum Royals won by 110 runs,None,None,None,None,None,None,None,None,None,None,None
4,132632,Uttar Pradesh Premier League 2025,Lucknow Falcons,Gaur Gorakhapur Lions,Lucknow Falcons opt to bat,None,None,None,None,None,None,None,None,None,None,None
5,132621,Uttar Pradesh Premier League 2025,Kashi Rudras,Meerut Mavericks,Kashi Rudras won by 5 runs,None,None,None,None,None,None,None,None,None,None,None
6,116489,"New Zealand A tour of South Africa, 2025",South Africa A,New Zealand A,New Zealand A need 98 runs in 91 balls,None,None,None,None,None,None,None,None,None,None,None
7,131625,ICC Womens T20 World Cup Africa Region Divisio...,Nigeria Women,Zimbabwe Women,Nigeria Women opt to bat,None,None,None,None,None,None,None,None,None,None,None
8,131614,ICC Womens T20 World Cup Africa Region Divisio...,Kenya Women,Uganda Women,Uganda Women opt to bowl,None,None,None,None,None,None,None,None,None,None,None
9,131609,ICC Womens T20 World Cup Africa Region Divisio...,Namibia Women,Sierra Leone Women,Namibia Women won by 152 runs,None,None,None,None,None,None,None,None,None,None,None


In [10]:
import sqlite3

# Connect (creates file if not exists)
conn = sqlite3.connect("cricbuzz.db")
cursor = conn.cursor()

# Drop old table if it exists
cursor.execute("DROP TABLE IF EXISTS live_matches;")

# Create new table
cursor.execute("""
CREATE TABLE live_matches (
    match_id TEXT PRIMARY KEY,
    series_name TEXT,
    team1 TEXT,
    team2 TEXT,
    status TEXT
);
""")

conn.commit()
conn.close()

print("✅ live_matches table created successfully!")

✅ live_matches table created successfully!


In [11]:
import sqlite3

rows = [
    ("131119", "Netherlands tour of Bangladesh 2025", "Bangladesh", "Netherlands", "Rain stops play"),
    ("122962", "Sri Lanka tour of Zimbabwe, 2025", "Zimbabwe", "Sri Lanka", "Sri Lanka won by 4 wkts"),
    ("130762", "Kerala Cricket League 2025", "Aries Kollam Sailors", "Kochi Blue Tigers", "Innings Break"),
    ("130756", "Kerala Cricket League 2025", "Adani Trivandrum Royals", "Alleppey Ripples", "Adani Trivandrum Royals won by 110 runs"),
    ("132632", "Uttar Pradesh Premier League 2025", "Lucknow Falcons", "Gaur Gorakhpur Lions", "Lucknow Falcons opt to bat"),
    ("132621", "Uttar Pradesh Premier League 2025", "Kashi Rudras", "Meerut Mavericks", "Kashi Rudras won by 5 runs"),
    ("116489", "New Zealand A tour of South Africa, 2025", "South Africa A", "New Zealand A", "New Zealand A need 98 runs in 91 balls"),
    ("131625", "ICC Womens T20 World Cup Africa Region Division", "Nigeria Women", "Zimbabwe Women", "Nigeria Women opt to bat"),
    ("131614", "ICC Womens T20 World Cup Africa Region Division", "Kenya Women", "Uganda Women", "Uganda Women opt to bowl"),
    ("131609", "ICC Womens T20 World Cup Africa Region Division", "Namibia Women", "Sierra Leone Women", "Namibia Women won by 152 runs"),
    ("131598", "ICC Womens T20 World Cup Africa Region Division", "Tanzania Women", "Rwanda Women", "Tanzania Women won by 18 runs")
]

conn = sqlite3.connect("cricbuzz.db")
cursor = conn.cursor()

cursor.executemany("INSERT INTO live_matches VALUES (?, ?, ?, ?, ?);", rows)

conn.commit()
conn.close()

print("✅ Sample rows inserted into live_matches!")

✅ Sample rows inserted into live_matches!


In [12]:
import pandas as pd
conn = sqlite3.connect("cricbuzz.db")
df = pd.read_sql("SELECT * FROM live_matches;", conn)
conn.close()
df

,match_id,series_name,team1,team2,status
0,131119,Netherlands tour of Bangladesh 2025,Bangladesh,Netherlands,Rain stops play
1,122962,"Sri Lanka tour of Zimbabwe, 2025",Zimbabwe,Sri Lanka,Sri Lanka won by 4 wkts
2,130762,Kerala Cricket League 2025,Aries Kollam Sailors,Kochi Blue Tigers,Innings Break
3,130756,Kerala Cricket League 2025,Adani Trivandrum Royals,Alleppey Ripples,Adani Trivandrum Royals won by 110 runs
4,132632,Uttar Pradesh Premier League 2025,Lucknow Falcons,Gaur Gorakhpur Lions,Lucknow Falcons opt to bat
5,132621,Uttar Pradesh Premier League 2025,Kashi Rudras,Meerut Mavericks,Kashi Rudras won by 5 runs
6,116489,"New Zealand A tour of South Africa, 2025",South Africa A,New Zealand A,New Zealand A need 98 runs in 91 balls
7,131625,ICC Womens T20 World Cup Africa Region Division,Nigeria Women,Zimbabwe Women,Nigeria Women opt to bat
8,131614,ICC Womens T20 World Cup Africa Region Division,Kenya Women,Uganda Women,Uganda Women opt to bowl
9,131609,ICC Womens T20 World Cup Africa Region Division,Namibia Women,Sierra Leone Women,Namibia Women won by 152 runs
